In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
def norm(X):
    X_stats = X.describe()
    X_stats = X_stats.transpose()
    return (X - X_stats['mean']) / X_stats['std']

In [ ]:
models_su = ["CNN_Script-Unrelated_Pz.h5", "CNN_Script-Unrelated_P4.h5", "CNN_Script-Unrelated_P3.h5", "CNN_Script-Unrelated_Fz.h5",
              "CNN_Script-Unrelated_F4.h5", "CNN_Script-Unrelated_F3.h5", "CNN_Script-Unrelated_Cz.h5", "CNN_Script-Unrelated_C4.h5", "CNN_Script-Unrelated_C3.h5"]

models_sr = ['CNN_Script-Related_Pz.h5','CNN_Script-Related_P4.h5','CNN_Script-Related_P3.h5','CNN_Script-Related_Fz.h5','CNN_Script-Related_F4.h5',
             'CNN_Script-Related_F3.h5','CNN_Script-Related_Cz.h5','CNN_Script-Related_C4.h5','CNN_Script-Related_C3.h5']

models_control = ['CNN_Control_Pz.h5','CNN_Control_P4.h5','CNN_Control_P3.h5','CNN_Control_Fz.h5',
                  'CNN_Control_F4.h5','CNN_Control_F3.h5','CNN_Control_Cz.h5','CNN_Control_C4.h5','CNN_Control_C3.h5']


cols = ["Fz", "Cz", "Pz", "C3", "F3", "C4", "P3", "F4", "P4", 'Timestamp']


###Put in own directory of own data 
data_test = pd.read_csv('Datasets/{}'.format("2019_Script-Related.csv"))

condition = input("What condition is used? (Control, Script-Related, Script-Unrelated)")

if condition == 'Control':
    model_list = models_control
elif condition == 'Script-Related':
    model_list = models_sr
elif condition == 'Script-Unrelated':
    model_list = models_su
else:
    raise TypeError("No appropriate condition entered")

electrode = input('Which electrode should be interpolated? (P3, Pz, P4, C3, Cz, C4, F3, Fz, F4)')

X_test = data_test[cols].drop(electrode, axis = 1)
y_test = data_test[electrode]

X_test_normed = norm(X_test)
X_test = np.reshape(X_test_normed, (X_test_normed.shape[0], X_test_normed.shape[1], 1))        

for e in model_list:
    if electrode in e:
        model = load_model(e)

y_pred = model.predict(X_test)      
y_pred = y_pred.reshape(-1,)

timestamps = data_test['Timestamp']
actual_means = y_test.groupby(timestamps).mean()
pred_means = pd.Series(y_pred, index = y_test.index).groupby(timestamps).mean()

mse = mean_squared_error(actual_means, pred_means)

plt.plot(actual_means.index, actual_means.values, label="Means of Actual Voltages")
plt.plot(pred_means.index, pred_means.values, label="Means of Predicted Voltagess", c="r")
plt.legend()
plt.title("{} {} MSE score: {:.2f}".format(condition, electrode, mse))
plt.xlabel("Timestamp (ms)")
plt.ylabel("Voltage (μV)".format(electrode))
ax = plt.gca()
ax.invert_yaxis()
plt.show()